In [ ]:
! pip install git+https://github.com/Simoniman/goes_api.git
! pip install git+https://github.com/Simoniman/himawari_api.git
! wget -N -O utils.py 'https://github.com/Simoniman/GeoSatStitch/raw/main/utils.py' 
! wget -N -O Himawari_Download.py 'https://github.com/Simoniman/GeoSatStitch/raw/main/Himawari_ahi/Himawari_Download.py'
! wget -N -O himawari_utils.py 'https://github.com/Simoniman/GeoSatStitch/raw/main/Himawari_ahi/himawari_utils.py'
! pip install patool
! pip install satpy
! pip install rasterio
! pip install matplotlib
! pip install cartopy 
! pip install eumdac
! pip install requests

In [ ]:
import os
import time
import datetime
import shutil
from glob import glob
import requests
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import eumdac
import patoolib
from satpy import Scene, MultiScene, DataQuery
from pyresample import create_area_def
from utils import find_format, find_reader, geo_area_def, extract_zip_files, read_credentials_from_config
import goes_api
from goes_api import download_latest_files
import himawari_api
from himawari_api import download_latest_files
import numpy as np
import xarray as xr
import sys
import subprocess
import time
import threading
from himawari_utils import *
from google.colab import drive


In [ ]:
if not os.path.exists('/content/drive/'):
    drive.mount('/content/drive/')

##### download seviri

In [ ]:
consumer_key, consumer_secret = read_credentials_from_config(filename='./personal_config.json')

credentials = (consumer_key, consumer_secret)

token = eumdac.AccessToken(credentials)

seviri_pdcid = "EO:EUM:DAT:MSG:HRSEVIRI" # prime data collection id
seviri_iodcid = "EO:EUM:DAT:MSG:HRSEVIRI-IODC" # indian ocean data collection id

datastore = eumdac.DataStore(token)
seviri_pdc = datastore.get_collection(seviri_pdcid) # prime data collection
seviri_iodc = datastore.get_collection(seviri_iodcid) # indian ocean data collection

seviri_pp_latest = seviri_pdc.search().first() # prime product_latest
seviri_iop_latest = seviri_iodc.search().first() # indian ocean product_latest
products = [seviri_pp_latest, seviri_iop_latest]

# Create the "tmp" directory if it doesn't exist
base_dir = './tmp/Meteosat_seviri'
os.makedirs(base_dir, exist_ok=True)

for count, product in enumerate(products):
    if os.path.exists(os.path.join(base_dir, str(count))):
      # If it exists, remove it and its contents
      shutil.rmtree(os.path.join(base_dir, str(count)))
    os.makedirs(os.path.join(base_dir, str(count)), exist_ok=True)
    try:
        with product.open() as fsrc, open(os.path.join(base_dir,str(count), os.path.basename(fsrc.name)), mode='wb') as fdst:
            shutil.copyfileobj(fsrc, fdst)
            print(f'Download of product {product} finished.')
            extract_zip_files(os.path.join(base_dir, str(count)))
    except eumdac.product.ProductError as error:
        print(f"Error related to the product '{product}' while trying to download it: '{error.msg}'")
    except requests.exceptions.ConnectionError as error:
        print(f"Error related to the connection: '{error.msg}'")
    except requests.exceptions.RequestException as error:
        print(f"Unexpected error: {error}")

print('All downloads are finished.')



##### download goes

In [ ]:
#### Define protocol and local directory
base_dir = "./tmp/"
protocol = "s3"
fs_args = {}

#### Define satellite, sensor, product_level and product
satellite = ["16","18"] # 16 (Goes 16) ==> EAST  /// 18 (Goes 18) ==> WEST
sensor = "ABI"
product_level = "L1B"
product = "Rad"

#### Define sector and filtering options
sector = "F"
scene_abbr = None  # DO NOT SPECIFY FOR FULL DISC SECTOR
scan_modes = None  # select all scan modes (M3, M4, M6)
channels = None  # select all channels
channels = ["C13"]  # select channels subset
filter_parameters = {}
filter_parameters["scan_modes"] = scan_modes
filter_parameters["channels"] = channels
filter_parameters["scene_abbr"] = scene_abbr

#### Downloading options
n_threads = 20  # n_parallel downloads
force_download = False  # whether to overwrite existing data on disk
look_ahead_minutes = 60
N = 1
operational_checks = True

# Create the base_dir if it doesn't already exist
if not os.path.exists(base_dir):
    os.makedirs(base_dir)
    print(f"Directory '{base_dir}' created successfully.")
else:
    print(f"Directory '{base_dir}' already exists.")


if type(satellite) == list:
  for sat in satellite : 
    fpaths = goes_api.download_latest_files(
        N=N,
        base_dir=base_dir,
        protocol=protocol,
        fs_args=fs_args,
        satellite=sat,
        sensor=sensor,
        product_level=product_level,
        product=product,
        sector=sector,
        filter_parameters=filter_parameters,
        n_threads=n_threads,
        force_download=force_download,
        check_data_integrity=True,
        progress_bar=True,
        verbose=True,
        look_ahead_minutes=look_ahead_minutes,
    )

else:
    fpaths = goes_api.download_latest_files(
        N=N,
        base_dir=base_dir,
        protocol=protocol,
        fs_args=fs_args,
        satellite=satellite,
        sensor=sensor,
        product_level=product_level,
        product=product,
        sector=sector,
        filter_parameters=filter_parameters,
        n_threads=n_threads,
        force_download=force_download,
        check_data_integrity=True,
        progress_bar=True,
        verbose=True,
        look_ahead_minutes=look_ahead_minutes,
    )

##### download himawari

In [ ]:
script_path = 'Himawari_Download.py' 
base_dir = "./tmp/"
channels = ["C13"]
gradual_unzip = True # False for skipping unzipping and 1 for unzipping
counter_list = []

# NOTE : in this case, the gradual_unzip argument must be True. 
# Create a threading Thread
t = threading.Thread(target=run_script_and_check, args=(script_path, base_dir, channels, True, counter_list))

# Start the thread
t.start()

# We should wait until first run of the main download function. using a wait function over a global list is much more efficient and reliable that waiting a specific amount of time using time.sleep function.
wait_function(counter_list)

##### Goes-18 scene & Goes-16 scene

In [ ]:
filenames = glob('./tmp/**/*.nc', recursive=True)

#G18
g18_idx = ['G18' in fn for fn in filenames].index(True)
g18_scn = Scene(reader='abi_l1b',filenames=[filenames[g18_idx]])
g18_scn.load(['C13'])

#G16
g16_idx = ['G16' in fn for fn in filenames].index(True)
g16_scn = Scene(reader='abi_l1b',filenames=[filenames[g16_idx]])
g16_scn.load(['C13'])

##### MSG-3 (Prime) scene & MSG-2 (Indian Ocean) scene 

In [ ]:
filenames = glob('./tmp/**/*.nat', recursive=True)

#MSG3
msg3_idx = ['MSG3' in fn for fn in filenames].index(True)
msg3_scn = Scene(reader='seviri_l1b_native', filenames=[filenames[msg3_idx]])
msg3_scn.load(['IR_108'])

#MSG2
msg2_idx = ['MSG2' in fn for fn in filenames].index(True)
msg2_scn = Scene(reader='seviri_l1b_native', filenames=[filenames[msg2_idx]])
msg2_scn.load(['IR_108'])

Himawari scene


In [ ]:
filenames = glob('./tmp/**/*.DAT', recursive=True)

him_scn = Scene(reader='ahi_hsd', filenames=filenames)
him_scn.load(['B13'])

##### create Multiscene object and resample

In [ ]:
# creating multiscene from list of scenes
scenes = [g18_scn, g16_scn, msg3_scn, msg2_scn, him_scn]
mscn = MultiScene(scenes)

# grouping
groups = { DataQuery(name='IR_group', wavelength=(10, 11, 12)) : ['B13', 'C13', 'IR_108'] }
mscn.group(groups)

# area definition
dst_area_def = create_area_def('world_area', 3857, 
                            # resolution=xr.DataArray([2000, 2000], attrs={"units": "meters"}),
                            shape=(4000, 5000), # (12515, 20037)
                            area_extent= [-180 + 1/1e9, -74, 180, 74], units='degrees')

# resampling
resampled = mscn.resample(dst_area_def, reduce_data=False, radius_of_influence=50000)

Without applying weight :

In [ ]:
blended_without_weights = resampled.blend() 

With applying weights :

In [ ]:
from satpy.modifiers.angles import get_satellite_zenith_angle,_get_sensor_angles
import pyresample
from pyresample import image, geometry
def _csatz_resamp_nparr(scene, band, area_def):
    # satz in geos coordinate system
    sata, satz = _get_sensor_angles(scene[band])
    csatz = np.cos(np.deg2rad(satz.to_numpy()))
    # we need to specify source and destination projection systems
    src_areadef = scene[band].attrs['area']
    dst_areadef = area_def
    # resample satz from source coordinate projection system to destination projection system
    csatz_con = image.ImageContainerNearest(csatz, src_areadef, radius_of_influence=50000) #pyresample.imagecontainer
    csatz_resamp_con = csatz_con.resample(dst_areadef) #pyresample.imagecontainer
    csatz_resamp_nparr = csatz_resamp_con.image_data #numpy.array
    return csatz_resamp_nparr

In [ ]:
gdrive_cashed_folder_name = 'cos_sat_zenith_3000-5000'
cache_path_g18 = f'./drive/MyDrive/{gdrive_cashed_folder_name}/csatz_resamp_nparr_g18.npy'
if os.path.isfile(cache_path_g18):
    csatz_resamp_nparr_g18 = np.load(cache_path_g18)
else:
    csatz_resamp_nparr_g18 = _csatz_resamp_nparr(scene=g18_scn, band='C13', area_def=dst_area_def)
    np.save(cache_path_g18, csatz_resamp_nparr_g18)

cache_path_g16 = f'./drive/MyDrive/{gdrive_cashed_folder_name}/csatz_resamp_nparr_g16.npy'
if os.path.isfile(cache_path_g16):
    csatz_resamp_nparr_g16 = np.load(cache_path_g16)
else:
    csatz_resamp_nparr_g16 = _csatz_resamp_nparr(scene=g16_scn, band='C13', area_def=dst_area_def)
    np.save(cache_path_g16, csatz_resamp_nparr_g16)

cache_path_msg3 = f'./drive/MyDrive/{gdrive_cashed_folder_name}/csatz_resamp_nparr_msg3.npy'
if os.path.isfile(cache_path_msg3):
    csatz_resamp_nparr_msg3 = np.load(cache_path_msg3)
else:
    csatz_resamp_nparr_msg3 = _csatz_resamp_nparr(scene=msg3_scn, band='IR_108', area_def=dst_area_def)
    np.save(cache_path_msg3, csatz_resamp_nparr_msg3)

cache_path_msg2 = f'./drive/MyDrive/{gdrive_cashed_folder_name}/csatz_resamp_nparr_msg2.npy'
if os.path.isfile(cache_path_msg2):
    csatz_resamp_nparr_msg2 = np.load(cache_path_msg2)
else:
    csatz_resamp_nparr_msg2 = _csatz_resamp_nparr(scene=msg2_scn, band='IR_108', area_def=dst_area_def)
    np.save(cache_path_msg2, csatz_resamp_nparr_msg2)

cache_path_him = f'./drive/MyDrive/{gdrive_cashed_folder_name}/csatz_resamp_nparr_him.npy'
if os.path.isfile(cache_path_him):
    csatz_resamp_nparr_him = np.load(cache_path_him)
else:
    csatz_resamp_nparr_him = _csatz_resamp_nparr(scene=him_scn, band='B13', area_def=dst_area_def)
    np.save(cache_path_him, csatz_resamp_nparr_him)

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.imshow(csatz_resamp_nparr_msg3)
plt.colorbar()

# it is possible to apply some more modifications to the weight function if required
# plt.figure()
# weight_log = np.log10(csatz_resamp_nparr_g16)+0.5
# weight_log_clipped = np.clip(x,-4,0)
# plt.imshow((weight_log_clipped+4)/4)
# plt.colorbar()

In [ ]:
from satpy.multiscene import stack
from functools import partial
weights = [csatz_resamp_nparr_g18, csatz_resamp_nparr_g16, csatz_resamp_nparr_msg3, csatz_resamp_nparr_msg2, csatz_resamp_nparr_him]
stack_with_weights = partial(stack, weights=weights)
blended_with_weights = resampled.blend(blend_function=stack_with_weights)


##### Visualization

In [ ]:
blended = blended_with_weights

In [ ]:
channel = 'IR_group'
crs = blended[channel].attrs['area'].to_cartopy_crs()
plt.figure(figsize=(20,12))

ax = plt.axes(projection=crs)

cmap = 'Greys'  # Example colormap without transparency

dataset = blended[channel]
dataset.plot.imshow(transform=crs, cmap=cmap)
ax.coastlines()
ax.gridlines(draw_labels=True, linewidth=1, color='gray', alpha=0.5, linestyle='--')

Or if you want to visualize in another software, you can save the dataset.

In [ ]:
blended.save_datasets()